# Ping Tester

In [1]:
from socket import *
from struct import pack, unpack
from os import getpid
from socket import AF_INET
from socket import SOCK_RAW
from time import time, sleep
import random
import threading
import select
import datetime, binascii

In [2]:
recvData = 0
timer = 0

In [3]:
def getChecksum(info):
    info = [(int(info[i:i+4],16)) for i in range(0, len(info), 4)]
    result = 0
    for x in info:
        result += x
        if result > 0xFFFF:
            result %= 0x10000
            result += 1
    result ^= 0xFFFF
    return result

In [4]:
def makeICMP():
    _type = '08'
    _code = '00'
    _checksum = '0000'
    _id = str(hex(getpid()))[2:]
    _seq = '000' + str(hex(random.randrange(1,15)))[2:]
    _realTime = hex(round(time()))[2:]
    _tempTime = '0000'
    _data = '08090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637'
    hex_data = _type + _code + _checksum + _id + _seq + _realTime + _tempTime + _data
    _checksum = hex(getChecksum(hex_data))[2:]
    result = _type + _code + _checksum + _id + _seq + _realTime + _tempTime + _data
    return result

In [5]:
def pong(wait_time):
    global timer, recvData
    icmp = getprotobyname("icmp")
    sock = socket(AF_INET, SOCK_RAW, icmp)
    get, _, _ = select.select([sock], [], [], wait_time)
    timer = time() - timer
    recvData = get

In [6]:
def ping(ipaddr):
    soc = socket(AF_INET,SOCK_RAW,1)
    packet = makeICMP()
    try:
        soc.sendto(bytes.fromhex(makeICMP()), (ipaddr, 1))
        return True
    except:
        print("ip주소 확인필요")
        return False

In [9]:
if __name__ == "__main__":
    global timer, recvData
    
    ip = '182.223.216.1'   # Input : Target IP
    send_count = 5         # Input : Send Count
    wait_time = 1          # Input : Wait Pong Time
    
    for i in range(send_count):
        reciver = threading.Thread(target=pong, args=(wait_time,))
        item = reciver.start()
        timer = time()
        if ping(ip) :
            sleep(wait_time)
            try:
                recvData = binascii.b2a_hex(recvData[0].recv(1024))
                if recvData[40:42] == b'00':
                    serverTime = int(recvData[56:64], 16) + wait_time
                    serverTime = datetime.datetime.fromtimestamp(serverTime).strftime('%Y-%m-%d %H:%M:%S')
                    print(ip, "\t\t ServerTime : ", serverTime, "\t 응답시간 : ", round(timer, 4), "sec")
                elif recvData[40:44] == b'0301':
                    addr_long = int(recvData[24:32],16)
                    print(ip, "\t\t ICMP - Destination Unreachable\t\t from :", inet_ntoa(pack(">L", addr_long)))
            except:
                print(ip, "\t\t 응답없음")

182.223.216.1 		 ServerTime :  2019-10-30 17:46:48 	 응답시간 :  0.0076 sec
182.223.216.1 		 ServerTime :  2019-10-30 17:46:49 	 응답시간 :  0.0069 sec
182.223.216.1 		 ServerTime :  2019-10-30 17:46:50 	 응답시간 :  0.0069 sec
182.223.216.1 		 ServerTime :  2019-10-30 17:46:51 	 응답시간 :  0.0063 sec
182.223.216.1 		 ServerTime :  2019-10-30 17:46:52 	 응답시간 :  0.0064 sec
